# Проект: Анализ базы данных сервиса для чтения книг по подписке.

**Описание проекта:**

Исследование проводится для стартап-компании, предлагающей клиентам сервис для чтения книг по подписке. Для анализа предоставлена база данных, содержащая информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

**Цель исследования:**

Провести анализ данных, выявить инсайты, которые помогут понять потребительские предпочтения.

**Описание данных:**

Таблица **books** cодержит данные о книгах:
* `book_id` — идентификатор книги;
* `author_id` — идентификатор автора;
* `title` — название книги;
* `num_pages` — количество страниц;
* `publication_date` — дата публикации книги;
* `publisher_id` — идентификатор издателя.

Таблица **authors** cодержит данные об авторах:
* `author_id` — идентификатор автора;
* `author` — имя автора.

Таблица **publishers** cодержит данные об издательствах:
* `publisher_id` — идентификатор издательства;
* `publisher` — название издательства;

Таблица **ratings** cодержит данные о пользовательских оценках книг:
* `rating_id` — идентификатор оценки;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, оставившего оценку;
* `rating` — оценка книги.

Таблица **reviews** содержит данные о пользовательских обзорах:
* `review_id` — идентификатор обзора;
* `book_id` — идентификатор книги;
* `username` — имя автора обзора;
* `text` — текст обзора.


In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine


In [ ]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

Создадим функцию, выводящую результат запроса:

In [2]:
def result(query):
     
    return pd.io.sql.read_sql(query, con = engine)

In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


## Выведем таблицы

In [4]:
query = '''SELECT * FROM books LIMIT 5'''
result(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query = '''SELECT COLUMN_NAME, DATA_TYPE
            FROM INFORMATION_SCHEMA.COLUMNS
            WHERE TABLE_NAME = 'books' AND COLUMN_NAME = 'publication_date' '''
result(query)

,column_name,data_type
0,publication_date,date


In [6]:
query = '''SELECT * FROM authors LIMIT 5'''
result(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
query = '''SELECT * FROM publishers LIMIT 5'''
result(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
query = '''SELECT * FROM ratings LIMIT 5'''
result(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
query = '''SELECT * FROM reviews LIMIT 5'''
result(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Узнаем, сколько книг вышло после 1 января 2000 года

In [10]:
query = '''SELECT COUNT(DISTINCT book_id) 
            FROM books 
            WHERE publication_date > '2000-01-01' '''
result(query)

,count
0,819


In [11]:
query = '''SELECT COUNT(*) 
            FROM books 
             '''
result(query)

,count
0,1000


В нашей базе данных 819 книг, вышедших после 1 января 2000 года. Как-то не очень много. Можно ли ее пополнить новыми бестселлерами и популярными классическими книгами? 
## Найдем количество обзоров и среднюю оценку для каждой книги  

In [13]:
query = '''SELECT b.book_id, 
                  b.title, 
                  COUNT(DISTINCT r.review_id) reviews,
                  AVG(rt.rating) avg_rating
            FROM books b
            LEFT JOIN reviews r ON b.book_id = r.book_id
            LEFT JOIN ratings rt ON b.book_id = rt.book_id
            GROUP BY b.book_id, title 
            ORDER BY reviews DESC, avg_rating DESC
            LIMIT 15'''
result(query)

,book_id,title,reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897
5,497,Outlander (Outlander #1),6,4.125000
6,750,The Hobbit or There and Back Again,6,4.125000
7,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
8,779,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
9,963,Water for Elephants,6,3.977273


Пользовательские обзоры написаны для 993 изданий. Наибольшее количество читательских отзывов у книг "Сумерки". Причем большое количество отзывов не гарантирует высокий средний рейтинг книги у читателей - сколько людей, столько и мнений. 
## Исключим из анализа брошюры и определим издательство, которое выпустило наибольшее число книг (толще 50 страниц)

In [14]:
query = ''' SELECT p.publisher, 
                   COUNT(b.book_id) AS book_count
            FROM books b
            JOIN publishers p ON b.publisher_id = p.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher
            ORDER BY book_count DESC
            LIMIT 1;'''
    
result(query)

,publisher,book_count
0,Penguin Books,42


Наибольшее число книг выпустило издательство 'Penguin Books'. Для формирования стратегии развития также будет полезно узнать, какое издательство самое популярное - выпускает книги с большим количеством пользовательских обзоров.
## Определим автора с самой высокой средней оценкой книг. Будем учитывать только книги с 50 и более оценками.

In [16]:
query = ''' SELECT a.author_id,
                   a.author,
                   AVG(rt.rating) AS average_rating
            FROM authors a
            JOIN books b ON a.author_id = b.author_id
            JOIN ratings rt ON b.book_id = rt.book_id
            WHERE b.book_id IN 
            (
                    SELECT book_id
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(rating_id) >= 50
                                )
            GROUP BY a.author_id, a.author
            ORDER BY average_rating DESC
            LIMIT 1'''
result(query)

,author_id,author,average_rating
0,236,J.K. Rowling/Mary GrandPré,4.287097


Самая высокая средняя оценка книг у писательницы Джоан Роулинг. Эта информация может помочь нам сформировать стратегию по контенту: на основе анализа популярных авторов можем разработать план по закупке новых книг.
## Найдем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [17]:
query = ''' WITH count_reviews AS(
                SELECT COUNT(review_id) count, username
                FROM reviews
                WHERE username IN 
                    
                    (                    
                    SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(ratings) > 48
                    )
                    
                GROUP BY username)
                    
                    SELECT AVG(count)
                    FROM count_reviews '''
    
result(query)

,avg
0,24.0


Покупатели книг часто просматривают обзоры на издания перед покупкой. Чем больше обзоров, тем выше вероятность, что книгой заинтересуются другие читатели. 

В нашей базе в среднем 24 обзора от пользователя, поставившего больше 48 оценок, это значит, что половина пользователей только оценивает книгу, но не рассказывает про нее свое мнение. Возможно, наличие поощрительной бонусной программы для пользователей может побудить их подробнее рассказать о своих впечатлениях о прочитанной книге.



## Вывод

- Для анализа была предоставлена база данных сервиса для чтения книг по подписке, содержащая информацию о 1000 книг. 80% книг в ней старше 2000 года. 

- Пользовательские обзоры написаны для 993 изданий. Наибольшее количество читательских отзывов у книг "Сумерки". Причем большое количество отзывов не гарантирует высокий средний рейтинг книги у читателей. Но, судя по топ-15 лидеров можно предположить, что читателям нравятся истории о фантастических мирах.

- Наибольшее число книг выпустило издательство 'Penguin Books'. Для формирования стратегии развития также будет полезно узнать, какое издательство самое популярное - выпускает книги с большим количеством пользовательских обзоров.

- Самая высокая средняя оценка книг у писательницы Джоан Роулинг. Эта информация может помочь нам сформировать стратегию по контенту: на основе анализа популярных авторов можем разработать план по закупке новых книг.

- В нашей базе в среднем 24 обзора от пользователя, поставившего больше 48 оценок, это значит, что половина пользователей только оценивает книгу, но не рассказывает про нее свое мнение. Возможно, наличие поощрительной бонусной программы для пользователей может побудить их подробнее рассказать о своих впечатлениях о прочитанной книге.

- Для того чтобы лучше понять потребительские предпочтения стоит также проанализировать отзывы и оценки пользователей пользователей, для понимания их ожиданий и выявления возможных недостатков в текущем предложении. 